# Calculating Metrics to Evaluate VS performance

This notebook applies the different metrics reviewed in the previous notebook to the VS scoring results and predictions given by different docking tools.

In [2]:
import pandas as pd
import numpy as np
import glob, os, sys
sys.path.append(r'..')

In [9]:
from modules.plotting_metrics import PlotMetric

## Loading Docking Results
> ### Docking Score

In [24]:
prot_name = 'cdk2'
patho_to_json_file = glob.glob((os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys_LIGS_INFO_LABELS_AUC_docking_DUD.json')))[0]
df_cdk2_labels = pd.read_json(patho_to_json_file)

In [7]:
# 
docking_tools = ['AD4_LE', 'AD4_LE', 'VINA', 'VINARDO']

# CSAR DockingScores
csar_dkScores = {dk_tool: pd.read_csv(F'../data/vs_docking_crys_ensemble_{dk_tool}.csv', index_col=0)
                 for dk_tool in docking_tools}


In [20]:
pdb_ids = csar_dkScores['VINA'].drop(['ActiveInactive'], axis = 1).columns

for pdb_id in pdb_ids:

Index(['1aq1', '1b38', '1b39', '1buh', '1ckp', '1di8', '1dm2', '1e1v', '1e1x',
       '1e9h',
       ...
       '6q4b', '6q4c', '6q4d', '6q4e', '6q4f', '6q4g', '6q4h', '6q4i', '6q4j',
       '6q4k'],
      dtype='object', length=402)